In [19]:
import taichi as ti
from ipycanvas import Canvas
from IPython.display import display
import ipywidgets as widgets
import time
import numpy as np
import signalflow as sf
from PIL import Image

# Initialize Taichi
ti.init(arch=ti.cpu)

################################################## Global Vars ##################################################


n = 500  # Image size (n x n)
img_path = "/Users/balintl/Documents/GitHub/sonification/experiments/lsm_paper/figures/cellular_seq/Timepoint_001_220518-ST_C03_s1.jpg"
padding = 40 # Padding around the image
refresh_interval = 1 / 60  # Minimum time between redraws (in seconds)




################################################## Data ##################################################


image = ti.Vector.field(3, dtype=float, shape=(n, n))  # RGB field for colors
bg = ti.Vector.field(3, dtype=float, shape=(n, n))  # Background image
probe = ti.Vector.field(3, dtype=float, shape=(n, n))  # probe

# Cursor position (normalized to [0, 1]) and mouse state
cursor_pos = ti.Vector.field(2, dtype=float, shape=())
mouse_pressed = ti.field(dtype=int, shape=())  # 1 if mouse button is pressed, 0 otherwise
probe_size = ti.field(dtype=float, shape=(2))  # Size of the probe (normalized)
square_stroke = 0.006  # Stroke width for the square

# read a background image into a numpy array
img = Image.open(img_path)
img = img.resize((n, n))
img_data = np.array(img)
img_data_norm = img_data / 255.0  # Normalize to [0, 1]
bg.from_numpy(img_data_norm)
bg_np = img_data


################################################## Kernels ##################################################


@ti.kernel
def draw_image():
    """Render the image and the probe over it."""
    center_x, center_y = cursor_pos[None]
    probe_w = probe_size[0]
    probe_h = probe_size[1]
    center_x = ti.max(ti.min(center_x, 1.0 - probe_w / 2), probe_w / 2)  # Clamp cursor x to width - probe width
    center_y = ti.max(ti.min(center_y, 1.0 - probe_h / 2), probe_h / 2)  # Clamp cursor y to height - probe height 
    color = ti.Vector([1.0, 0.0, 0.0]) if mouse_pressed[None] else ti.Vector([1.0, 1.0, 1.0])  # Red or white

    for i, j in image:
        x = i / image.shape[0]  # Normalize to [0, 1]
        y = j / image.shape[1]  # Normalize to [0, 1]
        dist_x = abs(x - center_x)
        dist_y = abs(y - center_y)
        if dist_x < probe_w / 2 and dist_y < probe_h / 2 and (dist_x > probe_w/2 - square_stroke or dist_y > probe_h/2 - square_stroke):
                image[i, j] = color  # Probe color
        else:
            image[i, j] = bg[i, j]  # Background image


@ti.kernel
def draw_probe():
    """Render the probe contents."""
    center_x, center_y = cursor_pos[None]
    probe_w = probe_size[0]
    probe_h = probe_size[1]
    center_x = ti.max(ti.min(center_x, 1.0 - probe_w / 2), probe_w / 2)  # Clamp cursor x to width - probe width
    center_y = ti.max(ti.min(center_y, 1.0 - probe_h / 2), probe_h / 2)  # Clamp cursor y to height - probe height

    for i, j in probe:
        x = i / image.shape[0]  # Normalize to [0, 1]
        y = j / image.shape[1]  # Normalize to [0, 1]
        if x < probe_w and y < probe_h:
            # get bg under square
            bg_x = i + int((center_x - (probe_w / 2)) * n)
            bg_y = j + int((center_y - (probe_h / 2)) * n)
            probe[i, j] = bg[bg_x, bg_y]
        else:
            probe[i, j] = ti.Vector([1.0, 1.0, 1.0]) # White


################################################## GUI ##################################################


# Create a canvas
canvas = Canvas(width=n*2 + padding*2, height=n + padding*2)
display(canvas)

# Create two sliders for controlling the Probe size
probe_w_slider = widgets.IntSlider(
    value=50,  # Initial value
    min=1,  # Minimum size
    max=n,   # Maximum size
    step=1, # Step size
    description='Probe Width',
)
display(probe_w_slider)

probe_h_slider = widgets.IntSlider(
    value=50,  # Initial value
    min=1,  # Minimum size
    max=n,   # Maximum size
    step=1, # Step size
    description='Probe Height',
)
display(probe_h_slider)

# Create a toggle button for audio
audio_graph_toggle = widgets.ToggleButton(
    value=False,
    description='Audio',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Toggle audio',
    icon="speaker"
)
display(audio_graph_toggle)


################################################## DSP ##################################################


graph = None
try:
    graph.destroy()
    graph = sf.AudioGraph(start=False)
except:
    graph = sf.AudioGraph(start=False)
    graph.clear()

def mix2samps(mixval, eps=1e-6):
    "Convert a mix value (used in sf.Smooth) to samples"
    return np.ceil(np.log(eps) / np.log(mixval))

def samps2mix(samps, eps=1e-6):
    "Convert samples to a mix value (used in sf.Smooth)"
    return eps ** (1 / samps)

# Synth
class Theremin(sf.Patch):
    def __init__(self, frequency=440, amplitude=0.5, smooth_n_samps=24000):
        super().__init__()
        frequency = self.add_input("frequency", frequency)
        amplitude = self.add_input("amplitude", amplitude)
        freq_smooth = sf.Smooth(frequency, samps2mix(smooth_n_samps))
        amplitude_smooth = sf.Smooth(amplitude, samps2mix(smooth_n_samps))
        sine = sf.SineOscillator(freq_smooth)
        output = sf.StereoPanner(sine * amplitude_smooth, pan=0)
        self.set_output(output)

theremin = Theremin()
theremin.play()


################################################## Interaction ##################################################


def redraw():
    """Render new frames for all kernels, then update the HTML canvas with the results."""
    global is_drawing

    if is_drawing:
        return  # Skip if we're already drawing

    is_drawing = True
    try:
        # Render the image with the probe
        draw_image()
        img_data = (image.to_numpy() * 255).astype(np.uint8)  # Scale to [0, 255] and convert to uint8
        img_data = np.transpose(img_data, (1, 0, 2))  # Transpose to match the canvas shape
        canvas.put_image_data(img_data, padding, padding)

        # Render the probe contents
        draw_probe()
        probe_data = (probe.to_numpy() * 255).astype(np.uint8)  # Scale to [0, 255] and convert to uint8
        probe_data = np.transpose(probe_data, (1, 0, 2))  # Transpose to match the canvas shape
        canvas.put_image_data(probe_data, n+20+padding, padding)
    finally:
        is_drawing = False  # Reset drawing state


def handle_mouse_event(x, y, pressed: int = 0):
    """Handle mouse, compute probe features, update synth(s), and render kernels."""
    global latest_event_time

    # Normalize mouse coordinates to [0, 1]
    x_clamped = np.clip(x-padding, 0, n-1)
    y_clamped = np.clip(y-padding, 0, n-1)
    cursor_pos[None] = [x_clamped / n, y_clamped / n]

    # Update mouse state
    if pressed == 2:
        mouse_pressed[None] = 1
    elif pressed == 3:
        mouse_pressed[None] = 0
    
    # Drop excess events over the refresh interval
    current_time = time.time()
    if current_time - latest_event_time < refresh_interval and pressed < 2: # only skip if mouse is up
        return  # Skip if we are processing too quickly
    latest_event_time = current_time  # Update the last event time

    # Compute probe features
    # TODO: Implement probe features

    # Update synth
    mouse_x = np.clip(x_clamped / n, probe_size[0] / 2, 1 - probe_size[0] / 2)
    mouse_y = 1 - np.clip(y_clamped / n, probe_size[1] / 2, 1 - probe_size[1] / 2)
    theremin.set_input("frequency", 440 * 2 ** (mouse_y * 4 - 2))
    theremin.set_input("amplitude", mouse_x * mouse_pressed[None])
    
    # Compute all kernels
    redraw()

# GUI callbacks

# Update probe size from sliders
def update_probe_width(change):
    probe_size[0] = change['new'] / n
    redraw()

def update_probe_height(change):
    probe_size[1] = change['new'] / n
    redraw()

# Toggle DSP
def toggle_audio(change):
    if change['new']:
        graph.start()
    else:
        graph.stop()

# Mousing event listeners
canvas.on_mouse_move(lambda x, y: handle_mouse_event(x, y, 1 if mouse_pressed[None] == 1 else 0))  # Triggered during mouse movement
canvas.on_mouse_down(lambda x, y: handle_mouse_event(x, y, pressed=2))  # Mouse button pressed
canvas.on_mouse_up(lambda x, y: handle_mouse_event(x, y, pressed=3))  # Mouse button released

# GUI event listeners
probe_w_slider.observe(update_probe_width, names='value')
probe_h_slider.observe(update_probe_height, names='value')
audio_graph_toggle.observe(toggle_audio, names='value')


################################################## INIT ##################################################

# Global state variables
is_drawing = False
latest_event_time = time.time()
# Initial draw
image.fill(0)  # Clear the field
cursor_pos[None] = [0.5, 0.5]  # Start with the square in the center
mouse_pressed[None] = 0  # Start with mouse unpressed
probe_size[0], probe_size[1] = [probe_w_slider.value / n, probe_h_slider.value / n]
redraw()


[Taichi] Starting on arch=arm64


Canvas(height=580, width=1080)

IntSlider(value=50, description='Probe Width', max=500, min=1)

IntSlider(value=50, description='Probe Height', max=500, min=1)

ToggleButton(value=False, description='Audio', icon='speaker', tooltip='Toggle audio')

AudioGraph: The global audio graph has already been created. To create a new graph, call .destroy() first.
